In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'crashdata2021.csv'
crash_data = pd.read_csv(url, sep=',')
crash_data.head()

,CrashFactId,Name,MinorInjuries,ModerateInjuries,SevereInjuries,FatalInjuries,TcrNumber,CityDamageFlag,ShortFormFlag,Distance,...,PrimaryCollisionFactor,TrafficControl,Weather,CollisionType,ProximityToIntersection,VehicleInvolvedWith,PedestrianDirectionFrom,PedestrianDirectionTo,DirectionFromIntersection,Comment
0,649339,CR-0000085219,0,0,0,0,21-002-0253,False,False,285.0,...,Unknown,Controls Functioning,Clear,Rear End,Non-Related,Parked Vehicle,Not Applicable,Not Applicable,South Of,NaN
1,649368,CR-0000085220,0,0,0,0,21-001-0325,False,False,51.0,...,Unknown,No Controls Present/Factor,Clear,Broadside,Non-Related,Parked Vehicle,Not Applicable,Not Applicable,South Of,NaN
2,649369,CR-0000085222,0,0,0,0,21-004-9709,False,True,NaN,...,Unknown,Controls Functioning,Clear,Rear End,Related,Other Vehicle,Not Applicable,Not Applicable,South Of,NaN
3,649370,CR-0000085225,0,0,0,0,21-002-0483,False,False,0.0,...,Violation Driver 1,Controls Functioning,Clear,Broadside,Intersection,Other Vehicle,Not Applicable,Not Applicable,At,NaN
4,649371,CR-0000085226,0,0,0,0,21-003-4001,False,True,51.0,...,Unknown,Unknown,Unknown,Other,Non-Related,Parked Vehicle,Not Applicable,Not Applicable,West Of,NaN


In [3]:
url = 'vehiclecrashdata2021.csv'
vehicle_data = pd.read_csv(url, sep=',')
vehicle_data.head()

,CrashName,Name,Sex,Age,Speed,VehicleDamage,PartyCategory,Sobriety,VehicleDirection,MovementPrecedingCollision,PartyType,OtherAssociatedFactor,VehicleCount,ViolationCode,ViolationCodeDescription
0,CR-0000085197,ACV-0000174389,M,33,NaN,Unknown,Driver,Impairment Not Known,West,Stopped,Car,Unknown,1,Unknown,Unknown
1,CR-0000085197,ACV-0000174390,NaN,0,NaN,Unknown,Driver,Impairment Not Known,West,Making Right Turn,Car,Unknown,1,Unknown,Unknown
2,CR-0000085218,ACV-0000174429,M,19,NaN,Major,Driver,Had Not Been Drinking,East,Proceeding Straight,Car,Speeding,1,23103,Reckless Driving 2
3,CR-0000085218,ACV-0000174430,NaN,0,NaN,Moderate,Parked,Not Applicable,Not Applicable,Parked,Car,Unknown,1,Not Applicable,Not Applicable
4,CR-0000085218,ACV-0000174431,NaN,0,NaN,Moderate,Parked,Not Applicable,Not Applicable,Parked,Car,Unknown,1,Not Applicable,Not Applicable


In [ ]:
crash_data = crash_data.drop(['CrashFactId','TcrNumber', 'CityDamageFlag',
                'ShortFormFlag', 'Distance','PrimaryCollisionFactor', 'TrafficControl',
                'CollisionType','ProximityToIntersection', 'VehicleInvolvedWith', 'PedestrianDirectionFrom',
                'PedestrianDirectionTo', 'DirectionFromIntersection', 'Comment', 'PedestrianAction',
                'RoadwayCondition'], axis=1)
crash_data.rename( columns = ({'Name':'CrashName'}), inplace=True)

In [ ]:
vehicle_data['Speed'] = np.where((vehicle_data['ViolationCodeDescription']=='Speeding') | 
                               (vehicle_data['OtherAssociatedFactor']=='Speeding'), 'over speed limits', 'under speed limits')

vehicle_data = vehicle_data.drop(['Name','Sex','VehicleDamage','VehicleDirection','MovementPrecedingCollision',
                                  'PartyCategory', 'VehicleCount','ViolationCode','ViolationCodeDescription',
                                  'OtherAssociatedFactor'], axis=1)

In [ ]:
output = pd.merge(vehicle_data, crash_data, on='CrashName', how='inner')
output.head()

In [ ]:
temp = output[output['Age']!=0].copy()


bins = [1,15,20,30,40,50,60,70,80,100]
labels = ['1-15','16-20','21-30','31-40','41-50','51-60','61-70','71-80','>80']
temp.loc[:,'Age'] = pd.cut(temp['Age'],bins=bins,labels=labels)


temp2 = temp[(temp['RoadwaySurface']!='Unknown') & (temp['Weather']!='Unknown') 
             & (temp['Sobriety']!='Impairment Not Known') & (temp['Sobriety']!='Not Applicable')
             & (temp['PartyType']!='Unknown') & (temp['Lighting']!='Unknown') & (temp["Weather"] != "Other")]

temp2 = temp2.replace(to_replace = ['Had Been Drinking - Under Influence', 
                            'Had Been Drinking - Impairment Unknown', 'Had Been Drinking - Not Under Influence'],
              value = 'Had Been Drinking')

temp2 = temp2.replace(to_replace = ['Had Not Been Drinking'],
              value = 'Not Altered')

temp2 = temp2.replace(to_replace = ['Slippery (Muddy Oily etc.)'],
              value = 'Slippery')

temp2 = temp2.replace(to_replace = ['Dark - Street Light Not Functioning'],
              value = 'Dark - No Street Light')

temp2 = temp2.replace(to_replace = ['Bicycle', 'Pedestrian', 'Wheelchair', 'Skateboard', 'Other'],
              value = 'Vulnerable Users')

temp2 = temp2.replace(to_replace = ['Bus - Other', 'Bus - School'],
              value = 'Bus')

temp2 = temp2.replace(to_replace = ['Car With Trailer', 'Emergency Vehicle'],
              value = 'Car')

temp2 = temp2.replace(to_replace = ['Light Rail Vehicle'],
              value = 'Train')

temp2 = temp2.replace(to_replace = ['Motorcycle/Moped', 'Scooter Motorized'],
              value = 'Motorcycle')

temp2 = temp2.replace(to_replace = ['Semi Truck', 'Panel Truck'],
              value = 'Truck')

temp2['CrashDateTime'] = pd.to_datetime(temp2['CrashDateTime'], format='%m/%d/%Y %I:%M:%S %p').dt.hour
temp2['CrashDateTime'] = temp2['CrashDateTime'].astype(str).replace(to_replace = ['0'], value = '24')

bins = [1,6,12,17,21,24]
labels = ['01-06','07-12','13-17','18-21','22-24']
temp2.loc[:,'CrashDateTime'] = pd.cut(temp2['CrashDateTime'].astype(int),bins=bins,labels=labels)

temp2['Injuries'] = np.where ((temp2['ModerateInjuries']>0) |
                             (temp2['SevereInjuries']>0) | (temp2['FatalInjuries']>0), 1, 0)

temp2 = temp2.drop(["CrashName","MinorInjuries","ModerateInjuries","SevereInjuries","FatalInjuries"],axis=1)

temp2.rename( columns = ({'Sobriety':'RoadUserConditions'}), inplace=True)
temp2.rename( columns = ({'RoadwaySurface':'RoadConditions'}), inplace=True)
temp2.rename( columns = ({'CrashDateTime':'CrashTime'}), inplace=True)

temp2.shape
temp3 = temp2.dropna()

In [ ]:
temp3.shape
temp3

In [ ]:
temp3.to_csv(r'C:\Users\franc\Desktop\Unibo\Fundamentals of AI and knowledge representation\Project module 3\data.csv', index = False)